In [0]:
# Importe das libs
from fastai2.vision.all import *
from fastai2.basics import *
from fastai2.callback.all import *

In [2]:
# Recuperacao da base MNIST completo.
path = untar_data(URLs.MNIST)

In [0]:
#Vetor contendo o path de cada imagem
items= get_image_files(path)

In [0]:
# O metodo GrandparentSplitter divide a base considerando o nome dos diretorios passados como parametro. 
# Note que informamos os diretorios training e testing, que sao como os nossos dados estao divididos
# apos o download do repositorio do fastai
splits = GrandparentSplitter(train_name='training', valid_name='testing')
# O vetor items, que possuem os caminhos de todas as imagens, é passado como parametro para efetuar a divisao
# em dados de treinamento e teste
splits = splits(items)

In [0]:
# A classe Datasets cria linhas input e output a partir de items. Neste caso, o input é a imagem e o output é classe a qual a imagem pertence (de 0 a 9)  
# Note que Datasets, para cada linha, aplica a transformações em tfms (criacao da imagem, rotula a imagem com o seu nome de diretorio e tranformacao de string em id) e
# faz essa operacao apenas para os diretorios de split
dataset = Datasets(items, tfms=[[PILImageBW.create],[parent_label, Categorize]], splits=splits)

In [0]:
# Lista de transformacoes a serem aplciadas no dataloader 
# RandomCrop - corta a imagem randomicamente (data augmentation) com tamanho 28
# ToTensor() - tranforma para tensor
# Normalize() - normaliza para cada batch
tfms = [ToTensor(), RandomCrop(size=28)]
gpu_tfms = [IntToFloatTensor(), Normalize()]

In [0]:
# Cria o dataloader com batch size de 128 e com as transformações especificadas
dls = dataset.dataloaders(bs=128, after_item=tfms, after_batch=gpu_tfms)

In [0]:
# Camada de convolucao
def conv2(ni, nf):
    return ConvLayer(ni, nf, stride=2)

In [0]:
# Resnet block
class ResBlock(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = ConvLayer(nf,nf)
        self.conv2 = ConvLayer(nf,nf)
        
    def forward(self, x): return x + self.conv2(self.conv1(x))

In [0]:
# Funcao que realiza uma convolucao e uma passagem pela Resnet 
def conv_and_res(ni,nf): return nn.Sequential(conv2(ni, nf), ResBlock(nf))

In [0]:
# Rede neural 
net = nn.Sequential(
    conv_and_res(1, 8),
    conv_and_res(8, 16),
    conv_and_res(16, 32),
    conv_and_res(32, 16),
    conv2(16, 10),
    Flatten()
)

In [12]:
# Realiza treinamento de 8 epocas 
# Acuracia de 99,4%
learn = Learner(dls, net, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(8, lr_max=1e-1)

epoch,train_loss,valid_loss,accuracy,time
0,0.128206,0.098212,0.970500,01:01
1,0.093823,0.719418,0.760800,01:01
2,0.070811,0.055252,0.984200,01:00
3,0.065830,0.061681,0.980400,01:01
4,0.043902,0.037288,0.987300,01:00
5,0.028922,0.021322,0.993100,01:00
6,0.016825,0.020185,0.993300,01:00
7,0.009066,0.016424,0.994300,01:01


In [0]:
# Exporta modelo treinado
learn.export()